In [2]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from firebase_admin import credentials
from firebase_admin import firestore, firestore_async
import firebase_admin

credpath = 'scrape/web-scraping-67540-firebase-adminsdk-5qm46-d1a7fa09d5.json'

# initialize firebase

firebase_admin.initialize_app(credentials.Certificate(credpath))

db = firestore.client()

# reading data from firebase
data = {"name": "Los Angeles", "state": "CA", "country": "USA"}

# Add a new doc in collection 'cities' with ID 'LA'
# db.collection("cities").document("LA").set(data)

totalJobs = []

In [3]:
def Get_Data(jobs):
    if (len(jobs) > 0):
        # Link Image Company
        linkImageCompany = jobs[8].find('img').get('src')
        print(f'Link Image Company: {linkImageCompany}' + '\n')
        # Name Job
        nameJob = jobs[8].find('h3', class_='title')
        nameJob = nameJob.find('span').text
        print(f'Job Name: {nameJob}' + '\n')
        # Company Name
        companyJobName = jobs[8].find('a', class_='company').text
        print(f'Company Name: {companyJobName}' + '\n')
        # Detail Job Link
        detailJobLink = jobs[8].find('a').get('href')
        print(f'Detail Job Link: {detailJobLink}' + '\n')
        # Link to Company
        linkToCompany = jobs[8].find('a', class_='company').get('href')
        print(f'Link to Company: {linkToCompany}' + '\n')
        # Website
        website = 'TopCV'
        print(f'Website: {website}' + '\n')
        # Simple info
        infos = jobs[8].find('div', class_='label-content')
        infos = infos.find_all('label')
        simpleInfo = []
        for info in infos:
            temp = info.text
            if info.get('class') == ['address']:
                address = info.text
                print(f'Address: {address}' + '\n')
            if info.get('class') == ['deadline']:
                temp = info.text.replace('\n', '')
            simpleInfo.append(temp)
        print(f'Simple Info: {simpleInfo}' + '\n')
        # Job Description
        html_text = requests.get(detailJobLink).text
        soup = BeautifulSoup(html_text, 'lxml')
        Description = []
        jobDescription = soup.find('div', class_='job-data')
        titles = jobDescription.find_all('h3')
        contents = jobDescription.find_all('div', class_='content-tab')
        count = 0
        for content in contents:
            # in li tag will be in a new line
            if (content.find('li')):
                arrLi = ''
                liTags = content.find_all('li')
                for li in liTags:
                    if li == liTags[-1]:
                        arrLi += li.text
                    else:
                        arrLi += li.text + ', '
                Description.append({
                    'title': titles[count].text,
                    'content': arrLi
                })
            # in p have <br> tag will be in a new line
            elif (content.find('p')):
                arrP = ''
                pTags = content.find_all('p')
                for p in pTags:
                    # decompose <br> tag
                    for br in p.find_all("br"):
                        br.replace_with(", ")
                    arrP += p.text
            elif content.find('strong') != None:
                content.find('strong').decompose()
                Description.append({
                    'title': titles[count].text,
                    'content': content.text
                })
            else:
                Description.append({
                    'title': titles[count].text,
                    'content': content.text
                })
            count += 1
        print(f'Description: {Description}' + '\n')


In [ ]:
def Find_Jobs_In_TopCV():
    url = "https://www.topcv.vn/"
    search = "React Native"
    search = search.strip().replace(" ", "-").lower()
    # options = webdriver.ChromeOptions()
    # options.add_experimental_option('excludeSwitches', ['enable-logging'])
    # driver = webdriver.Chrome(options=options)

    url = url + "tim-viec-lam-"+search+"?page=1"

    # driver.get(url)

    # html_text = requests.get(url).text
    # soup = BeautifulSoup(html_text, 'lxml')

    try:
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'lxml')
    except:
        print("Không kết nối được server")
        return

    jobs = soup.find_all('div', class_='job-item-default job-ta')
    jobs_highlight = soup.find_all(
        'div', class_='job-item-default bg-highlight job-ta')

    Get_Data(jobs)
    Get_Data(jobs_highlight)
